In [ ]:
import pandas as pd

df = pd.read_csv(r'E:\uni\NLP group project\originals\gemini_7newsgroups.csv')
df = df[['Unnamed: 0', 'text', 'label20', 'labels_name20', 'label', 'div', 'name', 'result']]
df = df[['name','label', 'result']]
df[['label', 'result']] = df[['label', 'result']].apply(lambda x: x.str.lower())
df = df.iloc[:-1]
print(df)

      name       label  result
0      1.0  Technology     NaN
1      2.0     Science     NaN
2      3.0     Science     NaN
3      4.0     Science     NaN
4      5.0    Politics     NaN
..     ...         ...     ...
888  889.0       Sport     NaN
889  890.0    Vehicles     NaN
890  891.0    Politics     NaN
891  892.0       Sport     NaN
892  893.0     Science     NaN

[893 rows x 3 columns]


In [34]:
import difflib
from thefuzz import fuzz

def post_process(df, results):
      # Ensure the DataFrame has a 'name' and 'result' column
      string_list = ["religion", "politics", "sport", "technology", "vehicles", "science", "miscellaneous"]
        
      # Iterate through each result in the results list
      for result in results:
            similarity = []
            for string in string_list:
                  # Calculate similarity score using fuzz.partial_ratio
                  similarity.append([fuzz.partial_ratio(list(result.values())[0][0], string), string])
            
            # Get the string with the highest similarity score
            max_similarity = max(similarity)
            
            # If similarity score is greater than 60, update result in DataFrame
            if max_similarity[0] > 60:
                  df.loc[df['name'] == list(result.keys())[0], 'result'] = max_similarity[1]
                  
            else:
                  # If no strong match, set result to an empty string
                  df.loc[df['name'] == list(result.keys())[0], 'result'] = ""
        




In [35]:
from sklearn.metrics import normalized_mutual_info_score

def calculate_NMI(df):
    
      ground_truth = df['label']
      predictions = df['result']
      
      # Calculate Normalized Mutual Information Score
      nmi_score = normalized_mutual_info_score(ground_truth, predictions)
      print(f"Normalized Mutual Information Score: {nmi_score}")
      return nmi_score

In [36]:
import numpy as np
from collections import Counter

def calculate_purity(df):
    
      predicted_labels = np.array(df['result'])
      true_labels = np.array(df['label'])
      
      # Get unique clusters
      unique_clusters = np.unique(predicted_labels)
      
      # Total number of instances
      total_instances = len(true_labels)
      
      # Calculate the number of correctly classified instances in each cluster
      correctly_classified = 0
      for cluster in unique_clusters:
            # Get the true labels of instances in the current cluster
            indices_in_cluster = np.where(predicted_labels == cluster)[0]
            labels_in_cluster = true_labels[indices_in_cluster]
            
            # Determine the most common true label in this cluster
            majority_label_count = Counter(labels_in_cluster).most_common(1)[0][1]
            
            # Add the number of correctly classified instances in this cluster
            correctly_classified += majority_label_count
      
      # Calculate purity
      purity = correctly_classified / total_instances
      print(f"Purity: {purity}")
      return purity

In [37]:
print(calculate_NMI(df))
print(calculate_purity(df))

Normalized Mutual Information Score: 0.6438803485292871
0.6438803485292871
Purity: 0.7726763717805151
0.7726763717805151
